In [10]:
#import library
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

import os
import zipfile
import fnmatch
from PIL import Image
import numpy as np

In [11]:
import re

def load_image_from_zip(zip_path, img_path):
    with zipfile.ZipFile(zip_path, 'r') as zf:
        with zf.open(img_path) as file:
            img = Image.open(file)
            return img.convert("RGB")  # Ensure the image is in RGB format
        
class ZipImageFolder(datasets.ImageFolder):
    def __init__(self, zip_path, root, transform=None):
        self.zip_path = zip_path
        self.root = root
        self.transform = transform
        self.classes = ['0_real', '1_fake']
        self.img_paths = self._get_image_paths()

    def _get_image_paths(self):
        img_paths = []
        with zipfile.ZipFile(self.zip_path, 'r') as zf:
            for file_info in zf.infolist():
                name = file_info.filename
                if fnmatch.fnmatch(name, f"{self.root}/*.jpg"):
                    label = 0 if '0_real' in name.split('/')[1] else 1
                    img_paths.append((name, label))
        return img_paths

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        img_path, label = self.img_paths[index]
        img = load_image_from_zip(self.zip_path, img_path)
        print(f"Image dimensions: {np.array(img).shape}")
        if self.transform:
            img = self.transform(img)
        return img, label

In [12]:
def prepare_data(zip_path, batch_size, image_size):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    train_dir = "AIGC-Detection-Dataset/train"
    val_dir = "AIGC-Detection-Dataset/val"
    test_dir = "AIGC-Detection-Dataset/val"

    train_dataset = ZipImageFolder(zip_path, train_dir, transform=transform)
    val_dataset = ZipImageFolder(zip_path, val_dir, transform=transform)
    test_dataset = ZipImageFolder(zip_path, test_dir, transform=transform)
    print(f"Data prepared:\nTrain: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    
    print("Data loaded")
    return train_loader, val_loader, test_loader

In [13]:
def build_model():
    model = models.resnet50()
    model.fc = nn.Linear(model.fc.in_features, 1)  # Binary classification
    return model.to(DEVICE)

In [14]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

In [15]:
def validate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE).float().unsqueeze(1)
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()
            predicted = (outputs > 0.5).int()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss /= len(val_loader)
    accuracy = correct / len(val_loader.dataset)
    return val_loss, accuracy

In [16]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    best_val_loss = float('inf')
    patience = 3
    epochs_without_improvement = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        batch = 1
        print(f"Number of batches: {len(train_loader)}")
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE).float().unsqueeze(1)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            print(f"Batch {batch} complete, Loss: {loss.item():.4f}")
            batch += 1
            

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")
        # Validation
        val_loss, val_accuracy = validate_model(model, val_loader, criterion)
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

        # Early stopping: if validation loss hasn't improved, stop training
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
            # Save the best model weights (optional)
            save_model(model)
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f"Early stopping at epoch {epoch + 1} due to no improvement in validation loss.")
            break


In [17]:
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE).float().unsqueeze(1)
            outputs = torch.sigmoid(model(images))
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

In [18]:
if __name__ == "__main__":
    zip_path = "../AIGC-Detection-Dataset.zip"
    batch_size = 16
    image_size = 512
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {DEVICE}")
    train_loader, val_loader, test_loader = prepare_data(zip_path, batch_size, image_size)

    # Model building
    model = build_model()

    # Loss function and optimizer
    learning_rate = 0.001
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training
    num_epochs = 1
    train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)

    # Testing
    test_model(model, test_loader)

    # Save the model
    save_model(model, "nn.pth")

Device: cuda
Data prepared:
Train: 45000, Val: 5000, Test: 5000
Data loaded
Number of batches: 2812
Image dimensions: (248, 640, 3)
Image dimensions: (428, 640, 3)
Image dimensions: (512, 512, 3)
Image dimensions: (512, 512, 3)
Image dimensions: (228, 152, 3)
Image dimensions: (429, 640, 3)
Image dimensions: (512, 512, 3)
Image dimensions: (500, 335, 3)
Image dimensions: (512, 512, 3)
Image dimensions: (500, 332, 3)
Image dimensions: (354, 640, 3)
Image dimensions: (512, 512, 3)
Image dimensions: (512, 512, 3)
Image dimensions: (512, 512, 3)
Image dimensions: (512, 512, 3)
Image dimensions: (478, 640, 3)


KeyboardInterrupt: 